In [2]:
from shapely import MultiPolygon
from tilebox.datasets import Client

area = MultiPolygon(
    [
        (((11.26, 47.39), (11.12, 47.17), (11.37, 47.09), (11.69, 47.21), (11.64, 47.38), (11.26, 47.39)),),
    ]
)

client = Client()
datasets = client.datasets()
sentinel2_msi = datasets.open_data.copernicus.sentinel2_msi
data = sentinel2_msi.collection("S2A_S2MSI2A").query(
    temporal_extent=("2025-04-01", "2025-05-01"),
    spatial_extent=area,
    show_progress=True,
)

In [3]:
from tilebox.storage import CopernicusStorageClient
import os

storage_client = CopernicusStorageClient(access_key=os.getenv("COPERNICUS_ACCESS_KEY"), secret_access_key=os.getenv("COPERNICUS_SECRET_ACCESS_KEY"))
objects = storage_client.list_objects(data.isel(time=0))
objects


['DATASTRIP/DS_2APS_20250404T184313_S20250404T101510/MTD_DS.xml',
 'DATASTRIP/DS_2APS_20250404T184313_S20250404T101510/QI_DATA/FORMAT_CORRECTNESS.xml',
 'DATASTRIP/DS_2APS_20250404T184313_S20250404T101510/QI_DATA/GENERAL_QUALITY.xml',
 'DATASTRIP/DS_2APS_20250404T184313_S20250404T101510/QI_DATA/GEOMETRIC_QUALITY.xml',
 'DATASTRIP/DS_2APS_20250404T184313_S20250404T101510/QI_DATA/RADIOMETRIC_QUALITY.xml',
 'DATASTRIP/DS_2APS_20250404T184313_S20250404T101510/QI_DATA/SENSOR_QUALITY.xml',
 'GRANULE/L2A_T32TPT_A051099_20250404T101510/AUX_DATA/AUX_CAMSFO',
 'GRANULE/L2A_T32TPT_A051099_20250404T101510/AUX_DATA/AUX_ECMWFT',
 'GRANULE/L2A_T32TPT_A051099_20250404T101510/IMG_DATA/R10m/T32TPT_20250404T101041_AOT_10m.jp2',
 'GRANULE/L2A_T32TPT_A051099_20250404T101510/IMG_DATA/R10m/T32TPT_20250404T101041_B02_10m.jp2',
 'GRANULE/L2A_T32TPT_A051099_20250404T101510/IMG_DATA/R10m/T32TPT_20250404T101041_B03_10m.jp2',
 'GRANULE/L2A_T32TPT_A051099_20250404T101510/IMG_DATA/R10m/T32TPT_20250404T101041_B04_10m

In [9]:
storage_client.download_objects(data.isel(time=0), objects=[o for o in objects if ("B02_10m" in o) or ("B03_10m" in o) or ("B04_10m" in o)], output_dir='downloads')


GRANULE/L2A_T32TPT_A051099_20250404T101510/IMG_DATA/R10m/T32TPT_20250404T101041_B02_10m.jp2:   0%|          | …

GRANULE/L2A_T32TPT_A051099_20250404T101510/IMG_DATA/R10m/T32TPT_20250404T101041_B03_10m.jp2:   0%|          | …

GRANULE/L2A_T32TPT_A051099_20250404T101510/IMG_DATA/R10m/T32TPT_20250404T101041_B04_10m.jp2:   0%|          | …

PosixPath('downloads/Sentinel-2/MSI/L2A/2025/04/04/S2A_MSIL2A_20250404T101041_N0511_R022_T32TPT_20250404T184313.SAFE')

In [13]:
import rioxarray

b02 = rioxarray.open_rasterio('./downloads/Sentinel-2/MSI/L2A/2025/04/04/S2A_MSIL2A_20250404T101041_N0511_R022_T32TPT_20250404T184313.SAFE/GRANULE/L2A_T32TPT_A051099_20250404T101510/IMG_DATA/R10m/T32TPT_20250404T101041_B02_10m.jp2')
b03 = rioxarray.open_rasterio('./downloads/Sentinel-2/MSI/L2A/2025/04/04/S2A_MSIL2A_20250404T101041_N0511_R022_T32TPT_20250404T184313.SAFE/GRANULE/L2A_T32TPT_A051099_20250404T101510/IMG_DATA/R10m/T32TPT_20250404T101041_B03_10m.jp2')
b04 = rioxarray.open_rasterio('./downloads/Sentinel-2/MSI/L2A/2025/04/04/S2A_MSIL2A_20250404T101041_N0511_R022_T32TPT_20250404T184313.SAFE/GRANULE/L2A_T32TPT_A051099_20250404T101510/IMG_DATA/R10m/T32TPT_20250404T101041_B04_10m.jp2')


In [16]:
import xarray as xr

rgb_array = xr.concat([b04, b03, b02], dim='band')
rgb_array.rio.write_crs(b04.rio.crs, inplace=True)
output_file = "sentinel2_rgb.tif"
rgb_array.rio.to_raster(output_file)

### Custom datasets

Two types:
- time-series
- spatio-temporal

Can use this for
- auxiliary sensor data (weather stations, aeronet data, etc.)
- cataloging metadata (storing intermediate or final data products)

How to:
1. Define the data type at https://console.tilebox.com/datasets/my-datasets/create
2. Ingest the data (see https://docs.tilebox.com/guides/datasets/ingest#ingesting-data)
3. Query the data in the way you are used to with Open Data, typed and performant